Importing necessary libraries and API key for using Chat GPT

In [ ]:
import pandas as pd
import re

In [5]:
import openai
Preste_key = "Your API"
openai.api_key = Preste_key

Form requests that can generate us sentences with necessary intentions

In [6]:
request_transfert = f"écris 100 phrases qui contiennent l'intention de demander un transfert bancaire. N'inclus pas l'information supplémentaire. Seulement l'intention de demander un transfert bancaire" \
                        'Exemple de sortie: ["phrase_1", "phrase_2"]'
request_cancel = f"écris 100 phrases qui contiennent l'intention d'annuler la réservation d'une chambre d'hôtel. N'inclus pas l'information supplémentaire. Seulement l'intention d'annuler la réservation d'une chambre d'hôtel." \
                        'Exemple de sortie: ["phrase_1", "phrase_2"]'
request_bagage_perdu = f"écris 100 phrases qui contiennent l'intention de trouver un bagage perdu. N'inclus pas l'information supplémentaire. Seulement l'intention de trouver un bagage perdu." \
                        'Exemple de sortie: ["phrase_1", "phrase_2"]'
request_voyage = f"écris 100 phrases qui contiennent l'intention de suggérer un voyage. N'inclus pas l'information supplémentaire. Seulement l'intention de suggérer un voyage." \
                        'Exemple de sortie: ["phrase_1", "phrase_2"]'
request_hotel = f"écris 100 phrases qui contiennent l'intention de réserver une chambre d'hôtel. N'inclus pas l'information supplémentaire. Seulement l'intention de réserver une chambre d'hôtel." \
                        'Exemple de sortie: ["phrase_1", "phrase_2"]'
request_resto = f"écris 100 phrases qui contiennent l'intention de réserver une table au restaurant. N'inclus pas l'information supplémentaire. Seulement l'intention de réserver une table au restaurant." \
                        'Exemple de sortie: ["phrase_1", "phrase_2"]'
request_flight = f"écris 100 phrases qui contiennent l'intention de réserver un vol. N'inclus pas l'information supplémentaire. Seulement l'intention de réserver un vol." \
                        'Exemple de sortie: ["phrase_1", "phrase_2"]'
request_info_flight = f"écris 100 phrases qui contiennent l'intention de se renseigner sur un vol. N'inclus pas l'information supplémentaire. Seulement l'intention de se renseigner sur un vol." \
                        'Exemple de sortie: ["phrase_1", "phrase_2"]'
request_visa = f"écris 100 phrases qui contiennent l'intention de se renseigner sur le visa. N'inclus pas l'information supplémentaire. Seulement l'intention de se renseigner sur le visa." \
                        'Exemple de sortie: ["phrase_1", "phrase_2"]'
request_car = f"écris 100 phrases qui contiennent l'intention de louer un véhicule (voiture, vélo, scooter etc.). N'inclus pas l'information supplémentaire. Seulement l'intention de louer un véhicule (voiture, vélo, scooter etc.)." \
                        'Exemple de sortie: ["phrase_1", "phrase_2"]'
all_requests = [request_transfert, request_cancel, request_bagage_perdu, request_voyage, request_hotel, request_resto, request_flight, request_info_flight, request_visa, request_car]

Generate requests for Chat GPT

In [7]:
def generate_phrases(request):
    completion = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                            messages=[{"role": "user", "content": request}]
                                            )
                #print(completion.choices[0].message.content)

    tmp_res = completion.choices[0].message.content
    return tmp_res

Preprocessing of Chat GPT outputs

In [8]:
def preprocess_phrases(tmp_res):
    lst_of_sent = tmp_res.split("\n")
    lst_of_sent_nodigits = []
    for snt in lst_of_sent:
        result = ''.join([i for i in snt if not i.isdigit()])
        lst_of_sent_nodigits.append(result)
    #print(lst_of_sent_nodigits)
    lst_of_sentences = []
    for snt in lst_of_sent_nodigits:
        snt = snt.lstrip(".")
        lst_of_sentences.append(snt)
    return lst_of_sentences

Generate sentences with requested intents

In [15]:
def generate_all_phrases(all_requests):
    lst_phrases = []
    for request in all_requests:
        tmp_res = generate_phrases(request)
        lst_phrases.append(preprocess_phrases(tmp_res))
    return lst_phrases       

In [17]:
all_phrases = generate_all_phrases(all_requests)

Form request to generate labels and rationals

In [18]:
def generate_request(phrase):

    request = f"phrase: {phrase}.\n" \
                "définis l'intention de cette phrase. Exemple: transfert bancaire, bagage perdu, réservation d'hôtel, réservation de restaurant, réservation de vol, annulation de réservation, location de voiture, suggestion de voyage, visa international, état du vol" \
                "explique pourquoi tu as choisi cette intention. Exemple: j'ai défini cette intention parce que + explication.\n" \
                'output as a dictionary. No additional text. Use template\n' \
                '{"phrase":phrase, "label":intention, "explication":explication.'
    return request

Generate labels and rationals with Chat GPT

In [29]:
def generate_rationals(lst_of_sentences):
    #all_info_lst = []
    all_info_sent = []
    for lst in lst_of_sentences:
        for sent in lst:
            tmp_req = generate_request(lst)
                # print(tmp_req)
            completion = openai.ChatCompletion.create(
                  model="gpt-3.5-turbo",
                  messages=[
                    {"role": "user", "content": tmp_req}
                  ]
                )
            #print(completion.choices[0].message.content)

            tmp_res = completion.choices[0].message.content

            all_info_sent.append(tmp_res)
    #all_info_lst.append(all_info_sent)
    return all_info_sent

In [30]:
lst_labels_rationals = generate_rationals(all_phrases)

Dataset transformation in dataframe

In [66]:
def dataset_formation(lst_labels_rationals):
    lst_phrases = []
    lst_labels = []
    lst_rationals = []
    for dct in lst_labels_rationals:
        try:
            dct = eval(dct)
        except SyntaxError:
            #print(dct)
            continue
        for sent in dct["phrase"]:
            #print(sent)
            lst_phrases.append(sent)
            lst_labels.append(dct["label"])
            lst_rationals.append(dct["explication"])
    data = {"Phrases": lst_phrases, "Labels": lst_labels, "Rationals": lst_rationals}
    df = pd.DataFrame(data)
    return df

In [67]:
generated_dataset = dataset_formation(lst_labels_rationals)

Saving to csv

In [68]:
generated_dataset.to_csv("dataset.csv")

,Phrases,Labels,Rationals
0,Est-il possible de demander un transfert banc...,transfert bancaire,J'ai choisi cette intention car toutes les phr...
1,Pourrais-je obtenir les informations nécessai...,transfert bancaire,J'ai choisi cette intention car toutes les phr...
2,Pourriez-vous m'indiquer les étapes à suivre ...,transfert bancaire,J'ai choisi cette intention car toutes les phr...
3,Serait-il faisable de solliciter un transfert...,transfert bancaire,J'ai choisi cette intention car toutes les phr...
4,Pourriez-vous m'aider à procéder à un transfe...,transfert bancaire,J'ai choisi cette intention car toutes les phr...
...,...,...,...
215,Je souhaite louer un véhicule pour me déplacer...,location de voiture,J'ai choisi cette intention car toutes les phr...
216,J'aimerais réserver un véhicule pour mes vacan...,location de voiture,J'ai choisi cette intention car toutes les phr...
217,Je suis intéressé(e) par la location d'un moye...,location de voiture,J'ai choisi cette intention car toutes les phr...
218,Je prévois de louer une voiture pour mon dépla...,location de voiture,J'ai choisi cette intention car toutes les phr...


!NB: Note that you may have some additional preprocessing according to the output you may have from Chat GPT. For example: it can add some addiotional symbols that you don't need and may need to remove. Also Chat GPT can produce some API errors, so you may need to wait in order to have a good output.